In [179]:
import pandas as pd
from itertools import chain, combinations

file_path = '/content/2_VER.xlsx'
df = pd.read_excel(file_path)
ar1=[]
print(df)

is_horizontal = 'TiD' in df.columns and 'items' in df.columns and len(df.columns) == 2

if is_horizontal:
    print("The dataset is already horizontal.")
    df['items'] = df['items'].apply(lambda x: ','.join(sorted(set(x.split(',')))))

else:
    print("The dataset is Vertical.")
    df['transactions'] = df['transactions'].astype(str).str.split(',')

    horizontal_data = {}
    for index, row in df.iterrows():
        for item in row['transactions']:
            if item in horizontal_data:
                horizontal_data[item].append(row['items'])
            else:
                horizontal_data[item] = [row['items']]

    df = pd.DataFrame(columns=['TiD', 'items'])
    for key, value in horizontal_data.items():
      df = df.append({'TiD': key, 'items': ','.join(value)}, ignore_index=True)

    df = df.sort_values(by='TiD').reset_index(drop=True)
    df['items'] = df['items'].apply(lambda x: ','.join(sorted(set(x.split(',')))))

    print("\nHorizontal Dataset After Modification:")

no_rows=len(df)
print(df)

  items              transactions
0     A  T1,T2,T5,T6,T7,T8,T9,T10
1     B         T1,T4,T5,T6,T7,T8
2     C     T1,T2,T3,T5,T7,T9,T10
3     D        T1,T2,T3,T5,T6,T10
4     E           T1,T2,T3,T5,T10
5     F                     T1,T4
The dataset is Vertical.

Horizontal Dataset After Modification:
   TiD        items
0   T1  A,B,C,D,E,F
1  T10      A,C,D,E
2   T2      A,C,D,E
3   T3        C,D,E
4   T4          B,F
5   T5    A,B,C,D,E
6   T6        A,B,D
7   T7        A,B,C
8   T8          A,B
9   T9          A,C


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-179-45fbfd62062b>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'TiD': key, 'items': ','.join(value)}, ignore_index=True)
<ipython-input-179-45fbfd62062b>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'TiD': key, 'items': ','.join(value)}, ignore_index=True)
<ipython-input-179-45fbfd62062b>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

In [180]:
def frequent_itemsets(df,minsupp):
   # Counting occurrences of each item
    item_counts = {}
    for index, row in df.iterrows():
        items = row['items'].split(',')
        for item in items:
            if item in item_counts:
                item_counts[item] += 1
            else:
                item_counts[item] = 1

    items_to_remove = [item for item, count in item_counts.items() if count < minsupp]

    # Remove infrequent items in frequent item set
    for x in items_to_remove:
      del item_counts[x]

    def frequent_itemset(l1, min_support,n):
      l2 = {}

      # Generate all possible combinations of 2 items
      item_combinations = list(combinations(l1.keys(), n))

      # Iterate through combinations and count support
      for combo in item_combinations:
          support_count = 0
          for row in df['items']:
              if all(item in row for item in combo):
                  support_count += 1

          if support_count >= min_support:
              l2[combo] = support_count
      return l2

    i=2
    ar=[]
    # print(item_counts)
    print("L1")
    ar.append(item_counts)
    while True :
      l=frequent_itemset(item_counts, minsupp,i)
      if len(l)==0 :
        break
      ar.append(l)
      #print(l)
      i+=1
      print(f"L{i-1}")
    return ar

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [181]:
frequent_itemsets(df,3)


L1
L2
L3
L4


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[{'A': 8, 'B': 6, 'C': 7, 'D': 6, 'E': 5},
 {('A', 'B'): 5,
  ('A', 'C'): 6,
  ('A', 'D'): 5,
  ('A', 'E'): 4,
  ('B', 'C'): 3,
  ('B', 'D'): 3,
  ('C', 'D'): 5,
  ('C', 'E'): 5,
  ('D', 'E'): 5},
 {('A', 'B', 'C'): 3,
  ('A', 'B', 'D'): 3,
  ('A', 'C', 'D'): 4,
  ('A', 'C', 'E'): 4,
  ('A', 'D', 'E'): 4,
  ('C', 'D', 'E'): 5},
 {('A', 'C', 'D', 'E'): 4}]

In [182]:
def print_association_rules(freq_itemset):
    # Generate all possible combinations of antecedents and consequents
    all_rules = []
    for rule_size in range(1, len(freq_itemset)):
        for rule in combinations(freq_itemset, rule_size):
            antecedent = list(rule)
            consequent = list(set(freq_itemset) - set(rule))
            all_rules.append((antecedent, consequent))

    # Print the generated association rules
    for rule in all_rules:
        antecedent_str = ''.join(sorted(rule[0]))
        consequent_str = ''.join(sorted(rule[1]))
        print(f"{antecedent_str} -> {consequent_str}")
        sor=sorted(antecedent_str+consequent_str)
        sor2=sorted(antecedent_str)
        sor3=sorted(consequent_str)
        ar1.append((sor2,sor,sor3))

    return ar1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [183]:
def calc_association_rule(mins):
  z=frequent_itemsets(df,mins)
  for x in z :
    for i in x :
      if len(i)>1:
        print(i)
        association_rules = print_association_rules(i)
  return association_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [184]:
z=frequent_itemsets(df,3)

L1
L2
L3
L4


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [187]:
def is_strong_rule(min_conf):
    conf=0.0
    prob_x_y=0.0
    prob_x=0.0
    for i in ar1:
      x=len(i[0])-1
      fc=len(i[1])-1
      if x!=0:
        a = ','.join(i[0])
        string = a
        tuple_from_string = tuple(string.split(','))
        prob_x=z[x][tuple_from_string]
        if fc!=0:
         g = ','.join(i[1])
         stre = g
         t_f_s = tuple(stre.split(','))
         prob_x_y=z[fc][t_f_s]
        else:
         g = ''.join(i[1])
         prob_x_y=z[fc][g]
      else:
        a = ''.join(i[0])
        prob_x=z[x][a]
        if fc!=0:
         g = ','.join(i[1])
         stre = g
         t_f_s = tuple(stre.split(','))
         prob_x_y=z[fc][t_f_s]
        else:
         g = ''.join(i[1])
         prob_x_y=z[fc][g]
      conf=prob_x_y / prob_x
      if conf >= min_conf:
          print(f"Confidence({', '.join(i[0])} -> {', '.join(i[2])}) = {conf:.2f}")
          print("The rule is strong.")
      else:
          print(f"Confidence({', '.join(i[0])} -> {', '.join(i[2])}) = {conf:.2f}")
          print("The rule is not strong.")

min_confidence_threshold = 0.7  # Set your minimum confidence threshold here

is_strong_rule(min_confidence_threshold)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [186]:
def calculate_lift():
  lift=0.0
  prob_x_y=0.0
  prob_x=0.0
  prob_y=0.0
  for i in ar1:
    x=len(i[0])-1
    fc=len(i[1])-1
    zc=len(i[2])-1
    if x!=0:
      a = ','.join(i[0])
      string = a
      tuple_from_string = tuple(string.split(','))
      # print(tuple_from_string)
      prob_x=z[x][tuple_from_string]
    else:
      a = ''.join(i[0])
      # print(a)
      prob_x=z[x][a]


    if fc!=0:
      g = ','.join(i[1])
      stre = g
      t_f_s = tuple(stre.split(','))
    #  print(t_f_s)
      prob_x_y=z[fc][t_f_s]
    else:
      g = ''.join(i[1])
    #  print(g)
      prob_x_y=z[fc][g]

    if zc!=0:
      op = ','.join(i[2])
      stre = op
      t_f_s = tuple(stre.split(','))
    #  print(t_f_s)
      prob_y=z[zc][t_f_s]
    else:
      op = ''.join(i[2])
    #  print(g)
      prob_y=z[zc][op]

    probb_x=prob_x/no_rows
    probb_y=prob_y/no_rows
    probb_x_y=prob_x_y/no_rows
    lift=probb_x_y / (probb_x * probb_y)
    if lift > 1:
        correlation_measure = "Dependent, + Correlated"
    elif lift < 1:
        correlation_measure = "Dependent, - Correlated"
    else:
        correlation_measure = "Independent"
    print(f"Confidence({', '.join(i[0])} -> {', '.join(i[2])})")
    print(lift)
    print(correlation_measure)
    # # Calculate the probability of X
    # prob_X = df[Col_Name].apply(lambda row: all(item in row for item in item_X)).sum() / total_rows

    # # Calculate the probability of Y
    # prob_Y = df[Col_Name].apply(lambda row: all(item in row for item in item_Y)).sum() / total_rows

    # # Calculate the probability of X and Y occurring together
    # prob_X_and_Y = df[Col_Name].apply(lambda row: all(item in row for item in item_X + item_Y)).sum() / total_rows

    # # Calculate lift
    # lift = prob_X_and_Y / (prob_X * prob_Y)

    # if lift > 1:
    #     correlation_measure = "Dependent, + Correlated"
    # elif lift < 1:
    #     correlation_measure = "Dependent, - Correlated"
    # else:
    #     correlation_measure = "Independent"

    # return lift, correlation_measure

calculate_lift()



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
